# Exploratory Data Analysis
I am using this notebook to learn how gamblers behaviors are similar to those of investors.

## Define Libraries

In [3]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive, fixed, IntSlider, HBox, Layout, VBox

# Getting rid of the SettingWithCopyWarning: 
pd.options.mode.chained_assignment = None

## Upload Data

In [4]:
# Set working directory
path = '/Users/mau/Library/CloudStorage/Dropbox/Mac/Documents/Dissertation/Chapter 2/Data'
os.chdir(path)

# Load data into a DataFrame
dtf = pd.read_parquet("df40.parquet")

print(dtf.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90000 entries, 0 to 90273
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   playercashableamt             90000 non-null  float64
 1   wageredamt                    90000 non-null  float64
 2   casino_grosswin               90000 non-null  float64
 3   playerkey                     90000 non-null  int64  
 4   age                           90000 non-null  int64  
 5   slotdenominationname          90000 non-null  object 
 6   maxbet                        90000 non-null  int64  
 7   assetnumber                   90000 non-null  int64  
 8   theoreticalpaybackpercent     90000 non-null  float64
 9   player_loss                   90000 non-null  float64
 10  player_wins                   90000 non-null  float64
 11  percent_return                90000 non-null  float64
 12  playercashableamt_pct_change  89823 non-null  float64
 13  t

In [5]:
# Print unique values of slotdenominationname
print(dtf['slotdenominationname'].unique())

# Delete rows were slotdenominationname is equal to 'Unknown Denomination'
dtf = dtf[dtf['slotdenominationname'] != 'Unknown Denomination']

# Print unique values of slotdenominationname
print(dtf['slotdenominationname'].unique())

# Create a new column called slotdenomination that is equal to the slotdenominationname column that do not have dollar signs and spaces
dtf['slotdenomination'] = dtf['slotdenominationname'].str.replace('$', '').str.replace(' ', '')

# Print unique values of slotdenomination
print(dtf['slotdenomination'].unique())

# Convert slotdenomination to float
dtf['slotdenomination'] = dtf['slotdenomination'].astype(float)

# Drop slotdenominationname column
dtf = dtf.drop(columns=['slotdenominationname'])

print(dtf.info())

['$5.00 ' '$10.00 ' '$1.00 ' '$2.00 ' '$0.25 ' '$0.05 ' '$0.01 ' '$0.02 '
 '$0.10 ' '$0.50 ' '$25.00 ' '$100.00 ' 'Unknown Denomination' '$50.00 ']
['$5.00 ' '$10.00 ' '$1.00 ' '$2.00 ' '$0.25 ' '$0.05 ' '$0.01 ' '$0.02 '
 '$0.10 ' '$0.50 ' '$25.00 ' '$100.00 ' '$50.00 ']
['5.00' '10.00' '1.00' '2.00' '0.25' '0.05' '0.01' '0.02' '0.10' '0.50'
 '25.00' '100.00' '50.00']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 89996 entries, 0 to 90273
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   playercashableamt             89996 non-null  float64
 1   wageredamt                    89996 non-null  float64
 2   casino_grosswin               89996 non-null  float64
 3   playerkey                     89996 non-null  int64  
 4   age                           89996 non-null  int64  
 5   maxbet                        89996 non-null  int64  
 6   assetnumber                   89996 non-nu

/var/folders/vp/1skwx2kd29s4fxnxx7tt9r6w0000gn/T/ipykernel_5708/845957601.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dtf['slotdenomination'] = dtf['slotdenominationname'].str.replace('$', '').str.replace(' ', '')


### Change Recognition
It is time to see which palyers changed machines to either increas or decrease their minimum bets.  

* Define function to look for our desire chage.
* We are going to be using _slotdenominationname_ to observe the change
* Create a variable _change_ that is 1 everytime _slotdenomination_ changes, 0 otherwise for each player

In [6]:
# Count number of unique players
print("Total number of Slot Players:", dtf['playerkey'].nunique())

Total number of Slot Players: 83


In [7]:
# Create a colunm increase_bet that is 1 everytime slotdenomination increases 0 otherwise per player
dtf["increase_slotdeno"] = dtf.groupby("playerkey")["slotdenomination"].diff().fillna(0)
dtf["increase_slotdeno"] = dtf["increase_slotdeno"].apply(lambda x: 1 if x > 0 else 0)

# Create a list of players that change to a higher slotdenomination machine
players_increase_slot = dtf[dtf["increase_slotdeno"] == 1]["playerkey"].unique().tolist()

# Count the number times each player changes to a higher slotdenomination machine, and store it in a dictionary
players_increase_slot_count = {}
for player in players_increase_slot:
    players_increase_slot_count[player] = dtf[(dtf["playerkey"] == player) & (dtf["increase_slotdeno"] == 1)]["increase_slotdeno"].count()


# Print count of players that change to a higher slotdenomination machine
print("Count of players who increased their slotdeno:", len(players_increase_slot))

# Print the dictionary that contains the count of times each player changes to a higher slotdenomination machine
print("Count of times each player increases their slotdeno:", players_increase_slot_count)

# Print the player who changes to a higher slotdenomination machine the most
print("Player who changes to a higher slotdenomination the most:", max(players_increase_slot_count, key=players_increase_slot_count.get))

# Print speration line
print("------------------------------------------------------------------------------------------------------------------")

# Create a colunm decrease_bet that is 1 everytime slotdenomination decreases 0 otherwise per player
dtf["decrease_slot"] = dtf.groupby("playerkey")["slotdenomination"].diff().fillna(0)
dtf["decrease_slot"] = dtf["decrease_slot"].apply(lambda x: 1 if x < 0 else 0)

# Create a list of players that decrease their bet
players_decrease_slot = dtf[dtf["decrease_slot"] == 1]["playerkey"].unique().tolist()

# Count the number times a player changes to a lower demonination slot, and store it in a dictionary
players_decrease_slot_count = {}
for player in players_decrease_slot:
    players_decrease_slot_count[player] = dtf[(dtf["playerkey"] == player) & (dtf["decrease_slot"] == 1)]["decrease_slot"].count()

# Print count of players that change to a lower slotdenomination machine
print("Count of players who decreased their slotdeno:", len(players_decrease_slot))

# Print the dictionary that contains the count of times each player changes to a lower slotdenomination machine
print("Count of times each player decreases their slotdeno:", players_decrease_slot_count)

# Print the player who changes to a lower slotdenomination machine the most
print("Player who changes to a lower slotdenomination the most:", max(players_decrease_slot_count, key=players_decrease_slot_count.get))

# Print speration line
print("------------------------------------------------------------------------------------------------------------------")

# Create a column called change_machine that is 1 everytime a player changes assetnumber 0 otherwise
dtf["change_machine"] = dtf.groupby("playerkey")["assetnumber"].diff().fillna(0)
dtf["change_machine"] = dtf["change_machine"].apply(lambda x: 1 if x != 0 else 0)

# Create a list of players that change machines
players_change_machine = dtf[dtf["change_machine"] == 1]["playerkey"].unique().tolist()

# Count the number times a player changes machines, and store it in a dictionary
players_change_machine_count = {}
for player in players_change_machine:
    players_change_machine_count[player] = dtf[(dtf["playerkey"] == player) & (dtf["change_machine"] == 1)]["change_machine"].count()

# Print count of players that change machines
print("Count of players who change machines:", len(players_change_machine))

# Print the dictionary that contains the count of times each player changes machines
print("Count of times each player changes machines:", players_change_machine_count)

# Print the player who changes machines the most
print("Player who changes machines the most:", max(players_change_machine_count, key=players_change_machine_count.get))


Count of players who increased their slotdeno: 50
Count of times each player increases their slotdeno: {2: 2, 3: 1, 4: 2, 6: 1, 7: 1, 8: 3, 9: 1, 11: 2, 12: 2, 14: 20, 17: 1, 18: 158, 19: 2, 20: 117, 27: 2, 29: 25, 33: 17, 35: 4, 36: 1, 37: 2, 38: 6, 40: 1, 41: 1, 43: 1, 44: 1, 47: 3, 48: 1, 49: 1, 51: 1, 54: 1, 56: 1, 57: 2, 61: 2, 68: 2, 69: 2, 70: 2, 73: 31, 76: 15, 79: 5, 83: 1, 84: 1, 85: 3, 87: 1, 89: 1, 91: 2, 92: 1, 93: 4, 94: 4, 95: 6, 97: 3}
Player who changes to a higher slotdenomination the most: 18
------------------------------------------------------------------------------------------------------------------
Count of players who decreased their slotdeno: 50
Count of times each player decreases their slotdeno: {2: 3, 3: 1, 4: 2, 6: 1, 8: 3, 9: 1, 11: 1, 12: 1, 14: 23, 18: 156, 19: 3, 20: 114, 27: 1, 29: 25, 30: 2, 33: 19, 35: 2, 37: 2, 38: 5, 43: 1, 44: 1, 47: 1, 48: 1, 49: 1, 51: 1, 53: 1, 54: 2, 61: 3, 65: 1, 68: 2, 69: 2, 70: 1, 72: 3, 73: 29, 76: 12, 79: 7, 83: 1, 85

In [8]:
# Create same code as above but for maxbet
# Create a colunm increase_bet that is 1 everytime maxbet increases 0 otherwise per player
dtf["increase_maxbet"] = dtf.groupby("playerkey")["maxbet"].diff().fillna(0)
dtf["increase_maxbet"] = dtf["increase_maxbet"].apply(lambda x: 1 if x > 0 else 0)

# Create a list of players that increase their bet
players_increase_maxbet = dtf[dtf["increase_maxbet"] == 1]["playerkey"].unique().tolist()

# Count the number of times each player increases their bet, and store this in a dictionary
players_increase_maxbet_count = {}
for player in players_increase_maxbet:
    players_increase_maxbet_count[player] = dtf[(dtf["playerkey"] == player) & (dtf["increase_maxbet"] == 1)]["increase_maxbet"].count()

# Print the number of players that increase their bet
print("Count of players who increased their bet:", len(players_increase_maxbet))

# Print the dictionary that contains the number of times each player increases their bet
print("Players who increased their bet and the number of times they increase their bet:", players_increase_maxbet_count)

# Print the player that increases their bet the most and the number of times they increase their bet
print("Player who increases their bet the most:", max(players_increase_maxbet_count, key=players_increase_maxbet_count.get))

# Print speration line
print("------------------------------------------------------------------------------------------------------------------")

# Create a colunm decrease_bet that is 1 everytime maxbet decreases 0 otherwise per player
dtf["decrease_maxbet"] = dtf.groupby("playerkey")["maxbet"].diff().fillna(0)
dtf["decrease_maxbet"] = dtf["decrease_maxbet"].apply(lambda x: 1 if x < 0 else 0)

# Create a list of players that decrease their bet
players_decrease_maxbet = dtf[dtf["decrease_maxbet"] == 1]["playerkey"].unique().tolist()

# Count the number of times each player decreases their bet, and store this in a dictionary
players_decrease_maxbet_count = {}
for player in players_decrease_maxbet:
    players_decrease_maxbet_count[player] = dtf[(dtf["playerkey"] == player) & (dtf["decrease_maxbet"] == 1)]["decrease_maxbet"].count()

# Print the number of players that decrease their bet
print("Count of players who decreased their bet:", len(players_decrease_maxbet))

# Print the dictionary that contains the number of times each player decreases their bet
print("Players who decrease their bet and the number of times they decrease their bet:", players_decrease_maxbet_count)

# Print the player that decreases their bet the most
print("Player who decreases their bet the most:", max(players_decrease_maxbet_count, key=players_decrease_maxbet_count.get))

Count of players who increased their bet: 62
Players who increased their bet and the number of times they increase their bet: {2: 2, 3: 2, 4: 2, 6: 2, 8: 3, 11: 1, 12: 2, 13: 3, 14: 4, 16: 2, 17: 2, 18: 2, 19: 3, 20: 3, 21: 1, 22: 1, 23: 2, 27: 2, 29: 2, 30: 2, 33: 4, 35: 4, 36: 5, 37: 2, 38: 6, 43: 1, 44: 1, 47: 3, 48: 3, 49: 1, 51: 1, 52: 1, 53: 2, 54: 3, 61: 1, 62: 1, 63: 2, 65: 1, 66: 3, 68: 1, 69: 4, 70: 2, 72: 4, 73: 9, 76: 10, 77: 1, 79: 4, 82: 1, 83: 1, 84: 1, 85: 1, 87: 2, 89: 2, 90: 2, 91: 1, 93: 34, 94: 17, 95: 2, 96: 2, 97: 4, 99: 6, 100: 4}
Player who increases their bet the most: 93
------------------------------------------------------------------------------------------------------------------
Count of players who decreased their bet: 63
Players who decrease their bet and the number of times they decrease their bet: {3: 1, 4: 1, 6: 3, 7: 2, 8: 3, 11: 2, 12: 3, 13: 2, 14: 5, 16: 2, 17: 3, 18: 1, 19: 2, 20: 3, 21: 1, 22: 2, 23: 3, 27: 1, 33: 4, 35: 7, 36: 4, 37: 3, 38: 8,

### Change Recognition Part 2

In this section we are analazying the change of slot denominantion and maxbet when the previous returns were either positive or negative.

In [18]:
def count_players_inc_lossing(dtf, n_periods, column):
    # Create a column called inc_slot_lossing that is 1 if players_increase_slot is 1 and their percent_return was negative the previous n_periods sessions, 0 otherwise
    inc_lossing_col = dtf.groupby("playerkey")[column].shift(1).fillna(0)
    inc_lossing_col = inc_lossing_col.apply(lambda x: 1 if x > 0 else 0)
    for i in range(1, n_periods+1):
        inc_lossing_col &= dtf["percent_return"].shift(i).fillna(0).apply(lambda x: 1 if x < 0 else 0)
    col_name = f"{column}_lossing_{n_periods}"
    dtf[col_name] = inc_lossing_col

    # Create a list of players that increase their bet when they were losing the previous n_periods sessions
    players_inc_lossing = dtf[dtf[col_name] == 1]["playerkey"].unique().tolist()

    # Count the number times a player increases their bet when they were losing the previous n_periods sessions, and store it in a dictionary
    players_inc_lossing_count = {}
    for player in players_inc_lossing:
        players_inc_lossing_count[player] = dtf[(dtf["playerkey"] == player) & (dtf[col_name] == 1)][col_name].count()

    # Print count of players that increase their slot denomination when they were losing the previous n_periods sessions
    print("Count of players who increase their bet when they were losing the previous", n_periods, "sessions:", len(players_inc_lossing))

    # Print the dictionary that contains the count of times each player increases their slot denomination when they were losing the previous n_periods sessions
    print("Count of times each player increases their bet when they were losing the previous", n_periods, "sessions:", players_inc_lossing_count)

    # Print the player who increases their slot denomination when they were losing the previous n_periods sessions the most
    print("Player who increases their bet when they were losing the previous", n_periods, "sessions the most:", max(players_inc_lossing_count, key=players_inc_lossing_count.get))

    # Print speration line
    print("------------------------------------------------------------------------------------------------------------------")
    return players_inc_lossing

def count_players_inc_winning(dtf, n_periods, column):
    # Create a column called inc_slot_winning that is 1 if players_increase_slot is 1 and their percent_return was positive the previous n_periods sessions, 0 otherwise
    inc_winning_col = dtf.groupby("playerkey")[column].shift(1).fillna(0)
    inc_winning_col = inc_winning_col.apply(lambda x: 1 if x > 0 else 0)
    for i in range(1, n_periods+1):
        inc_winning_col &= dtf["percent_return"].shift(i).fillna(0).apply(lambda x: 1 if x > 0 else 0)
    col_name = f"{column}_winning_{n_periods}"
    dtf[col_name] = inc_winning_col

    # Create a list of players that increase their bet when they were winning the previous n_periods sessions
    players_inc_winning = dtf[dtf[col_name] == 1]["playerkey"].unique().tolist()

    # Count the number times a player increases their bet when they were winning the previous n_periods sessions, and store it in a dictionary
    players_inc_winning_count = {}
    for player in players_inc_winning:
        players_inc_winning_count[player] = dtf[(dtf["playerkey"] == player) & (dtf[col_name] == 1)][col_name].count()
    
    # Print count of players that increase their slot denomination when they were winning the previous n_periods sessions
    print("Count of players who increased their bet when they were winning the previous", n_periods, "sessions:", len(players_inc_winning))

    # Print the dictionary that contains the count of times each player increases slot denomination when they were winning the previous n_periods sessions
    print("Count of times each player increases their bet when they were winning the previous", n_periods, "sessions:", players_inc_winning_count)

    # Print the player who increases their slot denomination when they were winning the previous n_periods sessions the most
    print("Player who increases their bet when they were winning the previous", n_periods, "sessions the most:", max(players_inc_winning_count, key=players_inc_winning_count.get))

    # Print speration line
    print("------------------------------------------------------------------------------------------------------------------")
    return players_inc_winning

def count_players_change_lossing(dtf, n_periods, column):
    # Create a column called inc_slot_lossing that is 1 if players_increase_slot is 1 and their percent_return was negative the previous n_periods sessions, 0 otherwise
    inc_lossing_col = dtf.groupby("playerkey")[column].shift(1).fillna(0)
    inc_lossing_col = inc_lossing_col.apply(lambda x: 1 if x > 0 else 0)
    for i in range(1, n_periods+1):
        inc_lossing_col &= dtf["percent_return"].shift(i).fillna(0).apply(lambda x: 1 if x < 0 else 0)
    col_name = f"{column}_lossing_{n_periods}"
    dtf[col_name] = inc_lossing_col

    # Create a list of players that increase their bet when they were losing the previous n_periods sessions
    players_inc_lossing = dtf[dtf[col_name] == 1]["playerkey"].unique().tolist()

    # Count the number times a player increases their bet when they were losing the previous n_periods sessions, and store it in a dictionary
    players_inc_lossing_count = {}
    for player in players_inc_lossing:
        players_inc_lossing_count[player] = dtf[(dtf["playerkey"] == player) & (dtf[col_name] == 1)][col_name].count()

    # Print count of players that increase their slot denomination when they were losing the previous n_periods sessions
    print("Count of players who changed machines when they were losing the previous", n_periods, "sessions:", len(players_inc_lossing))

    # Print the dictionary that contains the count of times each player increases their slot denomination when they were losing the previous n_periods sessions
    print("Count of times each player changed machines when they were losing the previous", n_periods, "sessions:", players_inc_lossing_count)

    # Print the player who increases their slot denomination when they were losing the previous n_periods sessions the most
    print("Player who changes machines when they were losing the previous", n_periods, "sessions the most:", max(players_inc_lossing_count, key=players_inc_lossing_count.get))

    # Print speration line
    print("------------------------------------------------------------------------------------------------------------------")
    return players_inc_lossing

def count_players_change_winning(dtf, n_periods, column):
    # Create a column called inc_slot_winning that is 1 if players_increase_slot is 1 and their percent_return was positive the previous n_periods sessions, 0 otherwise
    inc_winning_col = dtf.groupby("playerkey")[column].shift(1).fillna(0)
    inc_winning_col = inc_winning_col.apply(lambda x: 1 if x > 0 else 0)
    for i in range(1, n_periods+1):
        inc_winning_col &= dtf["percent_return"].shift(i).fillna(0).apply(lambda x: 1 if x > 0 else 0)
    col_name = f"{column}_winning_{n_periods}"
    dtf[col_name] = inc_winning_col

    # Create a list of players that increase their bet when they were winning the previous n_periods sessions
    players_inc_winning = dtf[dtf[col_name] == 1]["playerkey"].unique().tolist()

    # Count the number times a player increases their bet when they were winning the previous n_periods sessions, and store it in a dictionary
    players_inc_winning_count = {}
    for player in players_inc_winning:
        players_inc_winning_count[player] = dtf[(dtf["playerkey"] == player) & (dtf[col_name] == 1)][col_name].count()
    
    # Print count of players that increase their slot denomination when they were winning the previous n_periods sessions
    print("Count of players who changed machines when they were winning the previous", n_periods, "sessions:", len(players_inc_winning))

    # Print the dictionary that contains the count of times each player increases slot denomination when they were winning the previous n_periods sessions
    print("Count of times each player changes machines when they were winning the previous", n_periods, "sessions:", players_inc_winning_count)

    # Print the player who increases their slot denomination when they were winning the previous n_periods sessions the most
    print("Player who changes machines when they were winning the previous", n_periods, "sessions the most:", max(players_inc_winning_count, key=players_inc_winning_count.get))

    # Print speration line
    print("------------------------------------------------------------------------------------------------------------------")
    return players_inc_winning

Let's define the palyers who increase their slot denomination while lossing the previous sessions:

In [10]:
players_inc_slot_lossing_2 = count_players_inc_lossing(dtf, 2, "increase_slotdeno") # Look at previous 2 sessions
players_inc_slot_lossing_3 = count_players_inc_lossing(dtf, 3, "increase_slotdeno") # Look at previous 3 sessions
players_inc_slot_lossing_4 = count_players_inc_lossing(dtf, 4, "increase_slotdeno") # Look at previous 4 sessions

Count of players who increase their bet when they were losing the previous 2 sessions: 42
Count of times each player increases their bet when they were losing the previous 2 sessions: {2: 2, 3: 1, 4: 1, 8: 1, 11: 1, 12: 1, 14: 6, 18: 97, 20: 87, 27: 2, 29: 21, 33: 14, 35: 4, 36: 1, 37: 2, 38: 3, 40: 1, 41: 1, 43: 1, 47: 1, 48: 1, 49: 1, 51: 1, 54: 1, 57: 1, 61: 1, 68: 1, 69: 1, 70: 2, 73: 16, 76: 6, 83: 1, 84: 1, 85: 2, 87: 1, 89: 1, 91: 2, 92: 1, 93: 2, 94: 4, 95: 2, 97: 3}
Player who increases their bet when they were losing the previous 2 sessions the most: 18
------------------------------------------------------------------------------------------------------------------
Count of players who increase their bet when they were losing the previous 3 sessions: 39
Count of times each player increases their bet when they were losing the previous 3 sessions: {2: 2, 3: 1, 4: 1, 8: 1, 11: 1, 12: 1, 14: 6, 18: 86, 20: 69, 27: 2, 29: 18, 33: 14, 35: 4, 36: 1, 37: 2, 38: 2, 40: 1, 43: 1, 47: 

Let's define the players who increse their maxbet while loosing the previuos sessions

In [13]:
players_inc_maxbet_lossing_2 = count_players_inc_lossing(dtf, 2, "increase_maxbet") # Look at previous 2 sessions
players_inc_maxbet_lossing_3 = count_players_inc_lossing(dtf, 3, "increase_maxbet") # Look at previous 3 sessions
players_inc_maxbet_lossing_4 = count_players_inc_lossing(dtf, 4, "increase_maxbet") # Look at previous 4 sessions

Count of players who increase their bet when they were losing the previous 2 sessions: 57
Count of times each player increases their bet when they were losing the previous 2 sessions: {2: 1, 3: 1, 4: 2, 6: 2, 8: 1, 11: 1, 12: 2, 13: 3, 14: 2, 16: 2, 17: 1, 18: 2, 19: 1, 20: 2, 21: 1, 22: 1, 23: 2, 27: 2, 29: 2, 30: 1, 33: 1, 35: 2, 36: 2, 37: 1, 38: 2, 43: 1, 47: 2, 48: 2, 49: 1, 51: 1, 52: 1, 53: 1, 54: 3, 62: 1, 63: 1, 66: 2, 69: 2, 70: 1, 72: 3, 73: 4, 76: 5, 77: 1, 79: 2, 82: 1, 83: 1, 84: 1, 85: 1, 87: 1, 89: 2, 90: 1, 91: 1, 93: 27, 94: 16, 96: 2, 97: 1, 99: 5, 100: 4}
Player who increases their bet when they were losing the previous 2 sessions the most: 93
------------------------------------------------------------------------------------------------------------------
Count of players who increase their bet when they were losing the previous 3 sessions: 54
Count of times each player increases their bet when they were losing the previous 3 sessions: {2: 1, 3: 1, 4: 2, 6: 2, 8: 1

In [20]:
players_change_machine_lossing_2 = count_players_change_lossing(dtf, 2, "change_machine") # Look at previous 2 sessions
players_change_machine_lossing_3 = count_players_change_lossing(dtf, 3, "change_machine") # Look at previous 3 sessions
players_change_machine_lossing_4 = count_players_change_lossing(dtf, 4, "change_machine") # Look at previous 4 sessions

Count of players who changed machines when they were losing the previous 2 sessions: 69
Count of times each player changed machines when they were losing the previous 2 sessions: {2: 2, 3: 2, 4: 3, 6: 4, 8: 9, 9: 2, 11: 2, 12: 7, 13: 8, 14: 7, 16: 5, 17: 4, 18: 5, 19: 1, 20: 9, 21: 1, 22: 3, 23: 2, 27: 4, 29: 5, 30: 1, 33: 4, 35: 10, 36: 9, 37: 3, 38: 11, 40: 3, 43: 2, 44: 8, 46: 4, 47: 3, 48: 9, 49: 1, 51: 4, 52: 2, 53: 1, 54: 5, 56: 3, 61: 5, 62: 4, 63: 3, 65: 1, 66: 3, 69: 4, 70: 5, 72: 5, 73: 16, 76: 15, 77: 2, 79: 2, 82: 1, 83: 5, 84: 4, 85: 4, 86: 1, 87: 6, 88: 1, 89: 9, 90: 5, 91: 4, 92: 1, 93: 56, 94: 15, 95: 8, 96: 3, 97: 5, 98: 1, 99: 8, 100: 2}
Player who changes machines when they were losing the previous 2 sessions the most: 93
------------------------------------------------------------------------------------------------------------------
Count of players who changed machines when they were losing the previous 3 sessions: 68
Count of times each player changed machines wh

Let's define the players who increase their slot denomination while winning their previous sessions:

In [38]:
players_inc_slot_winning_2 = count_players_inc_winning(dtf, 2, "increase_slotdeno") # Look at previous 2 sessions
players_inc_slot_winning_3 = count_players_inc_winning(dtf, 3, "increase_slotdeno") # Look at previous 3 sessions
players_inc_slot_winning_4 = count_players_inc_winning(dtf, 4, "increase_slotdeno") # Look at previous 4 sessions

Count of players that increase their bet when they were winning the previous 2 sessions: 6
Count of times each player increases their bet when they were winning the previous 2 sessions: {6: 1, 14: 1, 18: 1, 19: 1, 57: 1, 73: 1}
Player who increases their bet when they were winning the previous 2 sessions the most: 6
------------------------------------------------------------------------------------------------------------------
Count of players that increase their bet when they were winning the previous 3 sessions: 1
Count of times each player increases their bet when they were winning the previous 3 sessions: {57: 1}
Player who increases their bet when they were winning the previous 3 sessions the most: 57
------------------------------------------------------------------------------------------------------------------
Count of players that increase their bet when they were winning the previous 4 sessions: 1
Count of times each player increases their bet when they were winning the pr

Let's define the palyers who increase their maxbet while winning the previous sessions:

In [39]:
players_inc_maxbet_winning_2 = count_players_inc_winning(dtf, 2, "increase_maxbet") # Look at previous 2 sessions
players_inc_maxbet_winning_3 = count_players_inc_winning(dtf, 3, "increase_maxbet") # Look at previous 3 sessions
# players_inc_maxbet_winning_4 = count_players_inc_winning(dtf, 4, "increase_maxbet") # Look at previous 4 sessions

Count of players that increase their bet when they were winning the previous 2 sessions: 3
Count of times each player increases their bet when they were winning the previous 2 sessions: {19: 1, 69: 1, 76: 1}
Player who increases their bet when they were winning the previous 2 sessions the most: 19
------------------------------------------------------------------------------------------------------------------
Count of players that increase their bet when they were winning the previous 3 sessions: 1
Count of times each player increases their bet when they were winning the previous 3 sessions: {69: 1}
Player who increases their bet when they were winning the previous 3 sessions the most: 69
------------------------------------------------------------------------------------------------------------------


In [19]:
players_change_machine_winning_2 = count_players_change_winning(dtf, 2, "change_machine") # Look at previous 2 sessions
players_change_machine_winning_3 = count_players_change_winning(dtf, 3, "change_machine") # Look at previous 3 sessions
# players_change_machine_winning_4 = count_players_inc_winning(dtf, 4, "change_machine") # Look at previous 4 sessions

Count of players who changed machines when they were winning the previous 2 sessions: 13
Count of times each player changes machines when they were winning the previous 2 sessions: {6: 1, 14: 2, 19: 1, 21: 1, 38: 1, 43: 1, 62: 1, 66: 1, 69: 1, 73: 1, 76: 1, 85: 1, 93: 1}
Player who changes machines when they were winning the previous 2 sessions the most: 14
------------------------------------------------------------------------------------------------------------------
Count of players who changed machines when they were winning the previous 3 sessions: 1
Count of times each player changes machines when they were winning the previous 3 sessions: {69: 1}
Player who changes machines when they were winning the previous 3 sessions the most: 69
------------------------------------------------------------------------------------------------------------------


## Slicing DataFrames per Matched Players and Visualizing Outcomes

By this point we have several lists of individuals who increased or decreased their bet by changing slot denomination or increasing their min bet. 

The first set of lists are (Generic):
* _players_increase_slot_
* _players_decrease_slot_
* _players_increase_maxbet_
* _players_decrease_maxbet_

The second set of lists are (While Lossing):
* _players_inc_slot_lossing_2_: players who change to a higher denomination slot while lossing their previous 2 sessions.
* _players_inc_slot_lossing_3_: players who change to a higher denomination slot while lossing their previous 3 sessions.
* _players_inc_slot_lossing_4_: players who change to a higher denomination slot while lossing their previous 4 sessions.
* _players_inc_maxbet_lossing_2_: players who increase their maxbet while lossing their previous 2 sessions.
* _players_inc_maxbet_lossing_3_: players who increase their maxbet while lossing their previous 3 sessions.
* _players_inc_maxbet_lossing_4_: players who increase their maxbet while lossing their previous 4 sessions.

The third set of lists are (While winning):
* _players_inc_slot_winning_2_: players who change to a higher denomination slot while winning their previous 2 sessions.
* _players_inc_slot_winning_3_: players who change to a higher denomination slot while winning their previous 3 sessions.
* _players_inc_slot_winning_4_: players who change to a higher denomination slot while winning their previous 4 sessions.
* _players_inc_maxbet_winning_2_: players who increase their maxbet while winning their previous 2 sessions.
* _players_inc_maxbet_winning_3_: players who increase their maxbet while winning their previous 2 sessions.

Let's define a slicing function for our lists to be used in the general dataframe.

In [40]:
def filter_match(df, players_match, match_column, rolling_window, fill_value):
    # Create a new DataFrame with only the players that appear in players_match
    df_match_all = df[df["playerkey"].isin(players_match)]

    # Creaete a new column for called match_rolling that is True for the rows around match_column is True
    df_match_all.loc[:, "match_rolling"] = df_match_all[match_column].rolling(window=rolling_window, center=True).apply(lambda x: any(x)).fillna(fill_value).astype(int)
    
    # Slice the DataFrame to only include the rows where match_rolling is True
    df_match_slice = df_match_all[df_match_all["match_rolling"] == True]
    # Return the new DataFrame
    return df_match_all, df_match_slice

First set of dataframes from generic lists

In [41]:
# Set rolling window (17 will set the rolling window to 17, 8 obersevations before and 8 observations after):
rolling = 17

# Create a new DataFrame with only the players that appear in players_increase_slot 
dtf_all_inc_slot, dtf_inc_slot = filter_match(df=dtf, players_match=players_increase_slot, match_column="increase_slotdeno", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_decrease_slot 
dtf_all_dec_slot, dtf_dec_slot = filter_match(df=dtf, players_match=players_decrease_slot, match_column="decrease_slot", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_increase_maxbet 
dtf_all_inc_maxbet, dtf_inc_maxbet = filter_match(df=dtf, players_match=players_increase_maxbet, match_column="increase_maxbet", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_decrease_maxbet 
dtf_all_dec_maxbet, dtf_dec_maxbet = filter_match(df=dtf, players_match=players_decrease_maxbet, match_column="decrease_maxbet", rolling_window=rolling, fill_value=False)

# Create a list of DataFrames that contains all the DataFrames that we want to plot
dtf_generic = [dtf_inc_slot, dtf_dec_slot, dtf_inc_maxbet, dtf_dec_maxbet]

Second set of dataframes from lossing previous rounds:

In [42]:
# Set rolling window (17 will set the rolling window to 17, 8 obersevations before and 8 observations after):
rolling = 17

# Create a new DataFrame with only the players that appear in players_inc_slot_lossing_2
dtf_all_inc_slot_lossing_2, dtf_inc_slot_lossing_2 = filter_match(df=dtf, players_match=players_inc_slot_lossing_2, match_column="increase_slotdeno_lossing_2", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_inc_slot_lossing_3
dtf_all_inc_slot_lossing_3, dtf_inc_slot_lossing_3 = filter_match(df=dtf, players_match=players_inc_slot_lossing_3, match_column="increase_slotdeno_lossing_3", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_inc_slot_lossing_4
dtf_all_inc_slot_lossing_4, dtf_inc_slot_lossing_4 = filter_match(df=dtf, players_match=players_inc_slot_lossing_4, match_column="increase_slotdeno_lossing_4", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_inc_maxbet_lossing_2
dtf_all_inc_maxbet_lossing_2, dtf_inc_maxbet_lossing_2 = filter_match(df=dtf, players_match=players_inc_maxbet_lossing_2, match_column="increase_maxbet_lossing_2", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_inc_maxbet_lossing_3
dtf_all_inc_maxbet_lossing_3, dtf_inc_maxbet_lossing_3 = filter_match(df=dtf, players_match=players_inc_maxbet_lossing_3, match_column="increase_maxbet_lossing_3", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_inc_maxbet_lossing_4
dtf_all_inc_maxbet_lossing_4, dtf_inc_maxbet_lossing_4 = filter_match(df=dtf, players_match=players_inc_maxbet_lossing_4, match_column="increase_maxbet_lossing_4", rolling_window=rolling, fill_value=False)

# Create a list of DataFrames that contains all the DataFrames that we want to plot
dtf_lossing = [dtf_inc_slot_lossing_2, dtf_inc_slot_lossing_3, dtf_inc_slot_lossing_4, dtf_inc_maxbet_lossing_2, dtf_inc_maxbet_lossing_3, dtf_inc_maxbet_lossing_4]

Third dataframes from winning previous rounds

In [43]:
# Set rolling window (17 will set the rolling window to 17, 8 obersevations before and 8 observations after):
rolling = 17

# Create a new DataFrame with only the players that appear in players_inc_slot_winning_2
dtf_all_inc_slot_winning_2, dtf_inc_slot_winning_2 = filter_match(df=dtf, players_match=players_inc_slot_winning_2, match_column="increase_slotdeno_winning_2", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_inc_slot_winning_3
dtf_all_inc_slot_winning_3, dtf_inc_slot_winning_3 = filter_match(df=dtf, players_match=players_inc_slot_winning_3, match_column="increase_slotdeno_winning_3", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_inc_slot_winning_4
dtf_all_inc_slot_winning_4, dtf_inc_slot_winning_4 = filter_match(df=dtf, players_match=players_inc_slot_winning_4, match_column="increase_slotdeno_winning_4", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_inc_maxbet_winning_2
dtf_all_inc_maxbet_winning_2, dtf_inc_maxbet_winning_2 = filter_match(df=dtf, players_match=players_inc_maxbet_winning_2, match_column="increase_maxbet_winning_2", rolling_window=rolling, fill_value=False)

# Create a new DataFrame with only the players that appear in players_inc_maxbet_winning_3
dtf_all_inc_maxbet_winning_3, dtf_inc_maxbet_winning_3 = filter_match(df=dtf, players_match=players_inc_maxbet_winning_3, match_column="increase_maxbet_winning_3", rolling_window=rolling, fill_value=False)

# Create a list of DataFrames that contains all the DataFrames that we want to plot
dtf_winning = [dtf_inc_slot_winning_2, dtf_inc_slot_winning_3, dtf_inc_slot_winning_4, dtf_inc_maxbet_winning_2, dtf_inc_maxbet_winning_3]

## Interactive Plots

The following section would be used to explore the data in an interactive way. These plots allow for user interaction, such as zooming, panning, and selecting data points. Users can customize the plot by choosing different variables to plot, adjusting axes ranges, and selecting data subsets. The interactive plots provide a dynamic way to visually explore the data and can reveal patterns or relationships that might not be apparent from static plots alone. By using interactive plots, we can gain a deeper understanding of the data and make more informed decisions during the data analysis process.

### First Plot Generic dataframes:

List of DataFrames that contains all generinc dataframes
* dtf_generic = [dtf_inc_slot, dtf_dec_slot, dtf_inc_maxbet, dtf_dec_maxbet]

In [44]:
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Make a list of all the dataframes that are match and slice
dtf_lists = dtf_generic

# Calculate the max and min values for the 'time' column for each DataFrame
time_max = max([df["time"].max() for df in dtf_lists])
time_min = min([df["time"].min() for df in dtf_lists])

print(time_max, time_min)

# Create a scatter plot of the players wins for only player with key 3
def plot_scatters(player_ID, df_index, x="time", y="percent_return", y_2=None, x_min=None, x_max=None, show_line=False, shade_area=False):
    df = dtf_lists[df_index]
    players = df["playerkey"].unique().tolist()
    player_df = df[df["playerkey"] == players[player_ID]]
    
    fig, ax1 = plt.subplots()
    ax1.set_xlabel(x)
    ax1.set_ylabel(y, color='royalblue')
    if x_min is not None and x_max is not None:
        player_df = player_df[(player_df[x] >= x_min) & (player_df[x] <= x_max)]
    ax1.scatter(x=player_df[x], y=player_df[y], color='royalblue')
    
    if y_2 is not None:
        ax2 = ax1.twinx()
        ax2.set_ylabel(y_2, color='r')
        if x_min is not None and x_max is not None:
            player_df = player_df[(player_df[x] >= x_min) & (player_df[x] <= x_max)]
        ax2.scatter(x=player_df[x], y=player_df[y_2], color='orangered', marker='s')
        ax2.tick_params(axis='y', labelcolor='orangered')
        # Add a line to the plot if show_line is True
        if show_line:
            ax2.plot(player_df[x], player_df[y_2], color='black', linewidth=0.8, linestyle='--')
        if shade_area:
            ax2.fill_between(player_df[x], player_df[y_2], color='lightcoral', alpha=0.5)
    
    if show_line:
        ax1.plot(player_df[x], player_df[y], color='black', linewidth=0.8)
        
    if shade_area:
        ax1.fill_between(player_df[x], player_df[y], color='lightblue', alpha=0.5)

    ax1.tick_params(axis='y', labelcolor='black')
    ax1.grid()
    plt.title(f"Player {players[player_ID]}")
    plt.show()

# Create widgets for playerkey, df_index, x, y, y_2, x_min, and x_max
df_index_widget = widgets.Dropdown(options=[(f"DataFrame {i}", i) for i in range(len(dtf_lists))])
x_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="time")
y_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="percent_return")
y_2_widget = widgets.Dropdown(options=[None]+list(dtf_lists[0].columns), value=None)
x_min_widget = widgets.FloatText(description="x_min", value=time_min)
x_max_widget = widgets.FloatText(description="x_max", value=time_max)
show_line_widget = widgets.Checkbox(description='Show line', value=False)
shade_area_widget = widgets.Checkbox(description='Shade area', value=False)

# Create a function to update the players_widget based on the selected df_index
def update_players_widget(df_index):
    df = dtf_lists[df_index]
    players = df["playerkey"].unique().tolist()
    player_key_widget.options = [(p, i) for i, p in enumerate(players)]

# Create a players_widget for the initial df_index value
initial_df_index = df_index_widget.value
initial_df = dtf_lists[initial_df_index]
initial_players = initial_df["playerkey"].unique().tolist()
player_key_widget = widgets.Dropdown(options=[], value=None)

# Call update_players_widget with the initial_df_index value to set the options for player_key_widget
update_players_widget(initial_df_index)

widgets.interact(plot_scatters, player_ID=player_key_widget, df_index=df_index_widget,
                 x=x_widget, y=y_widget, y_2=y_2_widget, x_min=x_min_widget, x_max=x_max_widget,
                 show_line=show_line_widget, shade_area=shade_area_widget)

# Update the player_key_widget options when df_index changes
def on_df_index_change(change):
    update_players_widget(change.new)

df_index_widget.observe(on_df_index_change, names='value')
update_players_widget(initial_df_index)  # update the player_key_widget options initially


8468 1


interactive(children=(Dropdown(description='player_ID', options=((2, 0), (3, 1), (4, 2), (6, 3), (7, 4), (8, 5…

### Second Plot Lossing Dataframes:

List of DataFrames that contains all lossing dataframes
* dtf_lossing = [dtf_inc_slot_lossing_2, dtf_inc_slot_lossing_3, dtf_inc_slot_lossing_4, dtf_inc_maxbet_lossing_2, dtf_inc_maxbet_lossing_3, dtf_inc_maxbet_lossing_4]

In [ ]:
# Make a list of all the dataframes that are match and slice
dtf_lists = dtf_lossing

# Calculate the max and min values for the 'time' column for each DataFrame
time_max = max([df["time"].max() for df in dtf_lists])
time_min = min([df["time"].min() for df in dtf_lists])

print(time_max, time_min)

# Create a scatter plot of the players wins for only player with key 3
def plot_scatters(player_ID, df_index, x="time", y="percent_return", y_2=None, x_min=None, x_max=None, show_line=False, shade_area=False):
    df = dtf_lists[df_index]
    players = df["playerkey"].unique().tolist()
    player_df = df[df["playerkey"] == players[player_ID]]
    
    fig, ax1 = plt.subplots()
    ax1.set_xlabel(x)
    ax1.set_ylabel(y, color='royalblue')
    if x_min is not None and x_max is not None:
        player_df = player_df[(player_df[x] >= x_min) & (player_df[x] <= x_max)]
    ax1.scatter(x=player_df[x], y=player_df[y], color='royalblue')
    
    if y_2 is not None:
        ax2 = ax1.twinx()
        ax2.set_ylabel(y_2, color='r')
        if x_min is not None and x_max is not None:
            player_df = player_df[(player_df[x] >= x_min) & (player_df[x] <= x_max)]
        ax2.scatter(x=player_df[x], y=player_df[y_2], color='orangered', marker='s')
        ax2.tick_params(axis='y', labelcolor='orangered')
        # Add a line to the plot if show_line is True
        if show_line:
            ax2.plot(player_df[x], player_df[y_2], color='black', linewidth=0.8, linestyle='--')
        if shade_area:
            ax2.fill_between(player_df[x], player_df[y_2], color='lightcoral', alpha=0.5)
    
    if show_line:
        ax1.plot(player_df[x], player_df[y], color='black', linewidth=0.8)
        
    if shade_area:
        ax1.fill_between(player_df[x], player_df[y], color='lightblue', alpha=0.5)

    ax1.tick_params(axis='y', labelcolor='black')
    ax1.grid()
    plt.title(f"Player {players[player_ID]}")
    plt.show()

# Create widgets for playerkey, df_index, x, y, y_2, x_min, and x_max
df_index_widget = widgets.Dropdown(options=[(f"DataFrame {i}", i) for i in range(len(dtf_lists))])
x_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="time")
y_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="percent_return")
y_2_widget = widgets.Dropdown(options=[None]+list(dtf_lists[0].columns), value=None)
x_min_widget = widgets.FloatText(description="x_min", value=time_min)
x_max_widget = widgets.FloatText(description="x_max", value=time_max)
show_line_widget = widgets.Checkbox(description='Show line', value=False)
shade_area_widget = widgets.Checkbox(description='Shade area', value=False)

# Create a function to update the players_widget based on the selected df_index
def update_players_widget(df_index):
    df = dtf_lists[df_index]
    players = df["playerkey"].unique().tolist()
    player_key_widget.options = [(p, i) for i, p in enumerate(players)]

# Create a players_widget for the initial df_index value
initial_df_index = df_index_widget.value
initial_df = dtf_lists[initial_df_index]
initial_players = initial_df["playerkey"].unique().tolist()
player_key_widget = widgets.Dropdown(options=[], value=None)

# Call update_players_widget with the initial_df_index value to set the options for player_key_widget
update_players_widget(initial_df_index)

widgets.interact(plot_scatters, player_ID=player_key_widget, df_index=df_index_widget,
                 x=x_widget, y=y_widget, y_2=y_2_widget, x_min=x_min_widget, x_max=x_max_widget,
                 show_line=show_line_widget, shade_area=shade_area_widget)

# Update the player_key_widget options when df_index changes
def on_df_index_change(change):
    update_players_widget(change.new)

df_index_widget.observe(on_df_index_change, names='value')
update_players_widget(initial_df_index)  # update the player_key_widget options initially


### Third Plot Winning Dataframes:

List of DataFrames that contains all winning dataframes:
* dtf_winning = [dtf_inc_slot_winning_2, dtf_inc_slot_winning_3, dtf_inc_slot_winning_4, dtf_inc_maxbet_winning_2, dtf_inc_maxbet_winning_3]

In [ ]:
# Make a list of all the dataframes that are match and slice
dtf_lists = dtf_winning

# Calculate the max and min values for the 'time' column for each DataFrame
time_max = max([df["time"].max() for df in dtf_lists])
time_min = min([df["time"].min() for df in dtf_lists])

print(time_max, time_min)

# Create a scatter plot of the players wins for only player with key 3
def plot_scatters(player_ID, df_index, x="time", y="percent_return", y_2=None, x_min=None, x_max=None, show_line=False, shade_area=False):
    df = dtf_lists[df_index]
    players = df["playerkey"].unique().tolist()
    player_df = df[df["playerkey"] == players[player_ID]]
    
    fig, ax1 = plt.subplots()
    ax1.set_xlabel(x)
    ax1.set_ylabel(y, color='royalblue')
    if x_min is not None and x_max is not None:
        player_df = player_df[(player_df[x] >= x_min) & (player_df[x] <= x_max)]
    ax1.scatter(x=player_df[x], y=player_df[y], color='royalblue')
    
    if y_2 is not None:
        ax2 = ax1.twinx()
        ax2.set_ylabel(y_2, color='r')
        if x_min is not None and x_max is not None:
            player_df = player_df[(player_df[x] >= x_min) & (player_df[x] <= x_max)]
        ax2.scatter(x=player_df[x], y=player_df[y_2], color='orangered', marker='s')
        ax2.tick_params(axis='y', labelcolor='orangered')
        # Add a line to the plot if show_line is True
        if show_line:
            ax2.plot(player_df[x], player_df[y_2], color='black', linewidth=0.8, linestyle='--')
        if shade_area:
            ax2.fill_between(player_df[x], player_df[y_2], color='lightcoral', alpha=0.5)
    
    if show_line:
        ax1.plot(player_df[x], player_df[y], color='black', linewidth=0.8)
        
    if shade_area:
        ax1.fill_between(player_df[x], player_df[y], color='lightblue', alpha=0.5)

    ax1.tick_params(axis='y', labelcolor='black')
    ax1.grid()
    plt.title(f"Player {players[player_ID]}")
    plt.show()

# Create widgets for playerkey, df_index, x, y, y_2, x_min, and x_max
df_index_widget = widgets.Dropdown(options=[(f"DataFrame {i}", i) for i in range(len(dtf_lists))])
x_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="time")
y_widget = widgets.Dropdown(options=list(dtf_lists[0].columns), value="percent_return")
y_2_widget = widgets.Dropdown(options=[None]+list(dtf_lists[0].columns), value=None)
x_min_widget = widgets.FloatText(description="x_min", value=time_min)
x_max_widget = widgets.FloatText(description="x_max", value=time_max)
show_line_widget = widgets.Checkbox(description='Show line', value=False)
shade_area_widget = widgets.Checkbox(description='Shade area', value=False)

# Create a function to update the players_widget based on the selected df_index
def update_players_widget(df_index):
    df = dtf_lists[df_index]
    players = df["playerkey"].unique().tolist()
    player_key_widget.options = [(p, i) for i, p in enumerate(players)]

# Create a players_widget for the initial df_index value
initial_df_index = df_index_widget.value
initial_df = dtf_lists[initial_df_index]
initial_players = initial_df["playerkey"].unique().tolist()
player_key_widget = widgets.Dropdown(options=[], value=None)

# Call update_players_widget with the initial_df_index value to set the options for player_key_widget
update_players_widget(initial_df_index)

widgets.interact(plot_scatters, player_ID=player_key_widget, df_index=df_index_widget,
                 x=x_widget, y=y_widget, y_2=y_2_widget, x_min=x_min_widget, x_max=x_max_widget,
                 show_line=show_line_widget, shade_area=shade_area_widget)

# Update the player_key_widget options when df_index changes
def on_df_index_change(change):
    update_players_widget(change.new)

df_index_widget.observe(on_df_index_change, names='value')
update_players_widget(initial_df_index)  # update the player_key_widget options initially
